In [1]:
import os

In [2]:
%pwd

'/home/mahi99/Documents/text_summarize_project/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/mahi99/Documents/text_summarize_project'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from textSummerizer.constants import *
from textSummerizer.utils.common import read_yaml,create_directories

In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARMS_FILE_PATH):


            self.config = read_yaml(config_filepath)
            #print("Config:", self.config)  # Debugging: Print the read configuration
            self.parms= read_yaml(params_filepath)
            #print("Params:", self.params)  # Debugging: Print the read parameters


            create_directories([self.config.artifacts_root])



    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_url= config.source_url,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        
        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
from textSummerizer.utils.common import get_size
from textSummerizer.logging import logger

In [9]:
class DataIngetion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    def Download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename= self.config.local_data_file
            )
            logger.info(f"{filename} download1! with following info:\n{headers}")
        else:
            logger.info(f"file already exits of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        if not zipfile.is_zipfile(self.config.local_data_file):
            raise zipfile.BadZipFile(f"File {self.config.local_data_file} is not a zip file")
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngetion(config=data_ingestion_config)
    data_ingestion.Download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-07-12 15:24:39,626: INFO: common: yaml file:/home/mahi99/Documents/text_summarize_project/config/config.yaml loaded successfully:]
[2024-07-12 15:24:39,630: INFO: common: yaml file:/home/mahi99/Documents/text_summarize_project/params.yaml loaded successfully:]
[2024-07-12 15:24:39,632: INFO: common: created directory at: artifacts:]
[2024-07-12 15:24:39,633: INFO: common: created directory at: artifacts/data_ingestion:]
[2024-07-12 15:24:39,635: INFO: 2173723168: file already exits of size: ~7718 KB:]
